In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from pylab import plt
plt.style.use('seaborn')
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'
import warnings; warnings.simplefilter('ignore')

## 지수 가져오기

In [ ]:
# get index vstoxx
url="http://www.stoxx.com/download/historical_values/h_vstoxx.txt"
#url= "https://www.stoxx.com/documents/stoxxnet/Documents/Indices/Current/HistoricalData/h_v2tx.txt"
vstoxx_index=pd.read_csv(url, index_col=0, header=2, parse_dates=True, dayfirst=True)
print(vstoxx_index.info())
vstoxx_index.tail()

## 선물데이터 가져오기 (나중에 파일 만들기)

In [ ]:
# get futures vstoxx
url="./Python_ex/resources/vstoxx_futures.xlsx"
vstoxx_futures=pd.read_excel(url, "vstoxx_futures")
print(vstoxx_futures.info())
del vstoxx_futures['A_SETTLEMENT_PRICE_SCALED']
del vstoxx_futures["A_PRODUCT_ID"]
del vstoxx_futures['SETTLE']
columns=['date','exp_year','exp_month','call_put_flag','price']
vstoxx_futures.columns=columns
vstoxx_futures.tail()



In [ ]:
V0 = 17.6639

In [ ]:
r = 0.01

In [ ]:
# 자료가 pandas DataFrame 객체로 되어 PyTable 데이터베이스 파일에 저장되어 있는 경우
import pandas as pd
import h5py
h5 = pd.HDFStore('./source/vstoxx_data_31032014.h5','r')
# VSTOXX 선물자료
futures_data = h5['futures_data']
# VSTOXX 콜 옵션 자료
options_data = h5['options_data']
h5.close()


In [ ]:
options_data.info()
options_data[['DATE', 'MATURITY','TTM','STRIKE','PRICE']].head()

## 콜옵션의 내재 변동성 계산

In [ ]:
# 선물 가격 기준 50% 이내의 행사가를 가진 옵션만을 다루도록 한다.
# 내재 변동성 추정 결과를 저장할 열 생성
options_data['IMP_VOL'] = 0.0

In [ ]:
# bsm function 불러오기
from bsm_functions import *

In [ ]:
# 내재 변동성 계산
tol = 0.5 #: 머니니스 범위
for option in options_data.index:
    # 모든 옵션 시장가에 대해 반복 작업
    forward = futures_data[futures_data['MATURITY'] == |
                          options_data.loc[option]['MATURITY']]['PRICE'].values[0]
       # 옵션과 만기가 일치하는 선물 선택
    if (forward * (1 - tol) < options_data.loc[option]['STRIKE']
                            < forward * (1 + tol)):
           #머니니스 범위 내의 옵션만 선택
            imp_vol = bsm_call_imp_col(
                    V0, # VSTOXX 지수값
                    options_data.loc[option]['STRIKE'],
                    options_data.loc[option]['TTM'],
                    r,
                    options_data.loc[option]['PRICE'],
                    sigma_est=2.,
                    it=100)
            options_data['IMP_VOL'].loc[option] = imp_vol

## 변동성 스마일 현상 확인

In [ ]:
''' options_data 중에서 내재 변동성이 계산 된 것만 이용한다'''
plot_data = options_data[options_data['IMP_VOL'] > 0]

In [ ]:
''' 같은 만기를 가진 자료를 모으기 위해 중복되지 않고 정렬된 상태의 만기 값리스트로 변환'''
maturities = sorted(set(options_data['MATURITY']))
# set 연산으로 중복제거, sorted로 정렬

In [ ]:
''' 각 만기별로 Plot 만들기'''
import matplotllib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8,6))
for maturity in maturities:
    data = plot_data[options_data.MATURITY == maturity]
      # 만기 maturity 값을 가진 자료만 선택
    plt.plot(data['STRIKE'], data['IMP_VOL'], 'r.')
    plt.grid(True)
    plt.xlabel('strike')
    plt.ylabel('implied volatility of volatility')
    plt.legend()
    plt.show()

## 계층 인덱스 기능 사용

In [ ]:
'''만기와 행사가를 통해 옵션을 선택할 경우, 만기와 행사가를 인덱스로 설정 가능'''
keep = ['PRICE', 'IMP_VOL']
group_data = plot_data.groupby(['MATURITY','STRIKE'])[keep]
group_data

In [ ]:
'''sum 을 통해 자료 찾기'''
group_data = group_data.sum()
group_data.head()
group_data.index.levels